<div class="alert alert-block alert-warning">
<b>Warning:</b> This notebook is deprecated. It will overwrite the Male meshes with the data from before the elbow UVs were moved. Instead use the data_manager.py module in the drawingto3D folder.
</div>

# Code to Save Mesh Data and Geodesic Solver
Saves out the data for the female and male meshes and the solvers for preloading of the data. This will greatly speed up calculations

In [1]:
import numpy as np
import polars as pl
import potpourri3d as pp3d

## Divide imported obj text file as a dataframe

In [2]:
def txt_to_dataframe(file_name):
    # Load in the text version of the model
    mesh_data =\
        pl.read_csv(f"../Models/{file_name}",
                    separator=' ', has_header=False,
                    new_columns=["Type", "Point 1", "Point 2", "Point 3"],
                    infer_schema_length=0)

    grouped_mesh_data = mesh_data.partition_by(["Type"], as_dict=True)

    # Get the UV data
    uv_data = grouped_mesh_data[("vt",)]
    uv_data = uv_data.with_columns(
        pl.col("Point 1").cast(pl.Float32)).with_columns(
            pl.col("Point 2").cast(pl.Float32)
        )
    uv_data = uv_data.drop("Type").drop("Point 3").with_row_index(offset=1)
    uv_data = uv_data.rename({"Point 1": "x", "Point 2": "y"})

    # Get the face data
    face_data = grouped_mesh_data[("f",)]
    split_face_data_1 = face_data.with_columns(
        pl.col("Point 1")
        .str.split_exact("/", 3)
        .struct.rename_fields(["vertex", "uv", "normal"])
        .alias("fields")
    ).unnest(
        "fields").drop("Type").drop("Point 1").drop("Point 2").drop("Point 3")

    split_face_data_2 = face_data.with_columns(
        pl.col("Point 2")
        .str.split_exact("/", 3)
        .struct.rename_fields(["vertex", "uv", "normal"])
        .alias("fields")
    ).unnest(
        "fields").drop("Type").drop("Point 1").drop("Point 2").drop("Point 3")

    split_face_data_3 = face_data.with_columns(
        pl.col("Point 3")
        .str.split_exact("/", 3)
        .struct.rename_fields(["vertex", "uv", "normal"])
        .alias("fields")
    ).unnest(
        "fields").drop("Type").drop("Point 1").drop("Point 2").drop("Point 3")

    split_face_data = pl.concat(
        [split_face_data_1, split_face_data_2, split_face_data_3])
    split_face_data = split_face_data.with_columns(
        pl.col("vertex").cast(pl.Int32)
    ).with_columns(
        pl.col("uv").cast(pl.UInt32)
    ).drop("normal").unique(subset="uv")

    return(uv_data, split_face_data)

## Function to save out data from mesh obj file

In [3]:
def get_mesh_data(mesh_name: str) -> None:
    '''
    Saves out the mesh data from an obj file

    Parameters
    ----------
    mesh_name: str
        The name of the mesh in the Model's folder
    '''
    # read in the mesh
    mesh_verticies, mesh_faces =\
        pp3d.read_mesh("../Models/" + mesh_name + " UV Mapped.obj")
    
    (uv_data, split_face_data) =\
        txt_to_dataframe(mesh_name + " UV Mapped as Text.txt")
    
    uv_array = uv_data.drop("index").to_numpy()
    lookup_data = split_face_data.to_numpy()

    np.savez_compressed("../Data/" + mesh_name.lower() + " mesh data.npz",
             mesh_verticies=mesh_verticies, mesh_faces=mesh_faces,
             uv_array=uv_array, lookup_data=lookup_data)

## Example for the Male Right Arm

In [4]:
mesh_name = "Male Right Arm"
get_mesh_data(mesh_name)

In [5]:
imported_data = np.load("../Data/male right arm mesh data.npz")
print(imported_data["uv_array"])

[[0.102348 0.273586]
 [0.931549 0.540414]
 [0.106193 0.042448]
 ...
 [0.151023 0.816322]
 [0.151804 0.817039]
 [0.152532 0.817768]]


## Face data needs to be imported and converted to a polars Dataframe

In [6]:
imported_lookup_data = pl.DataFrame(imported_data["lookup_data"],
                                    schema=["vertex", "uv"])
imported_lookup_data.head()

vertex,uv
i64,i64
5197,5377
141896,145369
106391,108546
151767,155480
191689,196894


## Save out the rest of the data

In [7]:
get_mesh_data("Male Left Arm")
get_mesh_data("Female Right Arm")
get_mesh_data("Female Left Arm")